In [1]:
import tweepy
import json
import pandas as pd
import csv

import mysql.connector
from mysql.connector import Error

#imports for catching the errors
from ssl import SSLError
from requests.exceptions import Timeout, ConnectionError
from urllib3.exceptions import ReadTimeoutError

In [2]:
#Twitter API credentials
consumer_key = "k6fUiadsPnsL9nsWi0kWNMsh4"
consumer_secret = "kpyZBrL0VHFN2SraS3Cwidi7Unxx7Vt0Um6oGvodlEGkRzAFBM"
access_token = "1234931384915021825-xjAmyTPHH1QPkXnuqGP0cwEUzYRwMx"
access_token_secret = "vHP7t1ce2KBYLAjAlZJve2XU1nhgSrGvJWrfOuChl2q2K"

In [3]:
#boundingbox ciudad de Madrid obtenido de https://boundingbox.klokantech.com/
madrid = [-3.7475842804,40.3721683069,-3.6409114868,40.4886258195] 
campus = [-3.7395722584,40.4342310784,-3.7156005182,40.4612470501] 
#Definicion palabras claves busqueda
keywords = ['descarbonización','descarbonizacion','clima','climático','climatico','combustible', 'CO2', 'climática', 'climatica', 'transición energética', 'renovable', 'energía', 'energia', 'energético', 'energética', 'energetico', 'energetica']

def connect(user_id, user_name, user_loc, user_follow_count,user_friends_count, user_fav_count,user_status_count,
            tweet_id,text,created_at,source,
            reply_id, reply_user_id,
            retweet_id,retweet_user_id,
            quote_id,quote_user_id,
            reply_count,retweet_count,favorite_count,quote_count, 
            hashtags, mention_ids,
            place_id, place_name, coord):

    con = mysql.connector.connect(host = 'localhost',
                                  database='twitterdb', user='david', password = 'password', charset = 'utf8mb4',auth_plugin='mysql_native_password')
    cursor = con.cursor()
    try:

        if con.is_connected():

            query = "INSERT INTO UsersCamCar (user_id, tweet_id,user_name, user_loc, user_follow_count,user_friends_count, user_fav_count,user_status_count) VALUES (%s,%s, %s, %s, %s, %s, %s, %s)"
            cursor.execute(query, (user_id, tweet_id, user_name, user_loc, user_follow_count,user_friends_count, user_fav_count,user_status_count))
            
               
            query2 = "INSERT INTO PostsCamCar (tweet_id,user_id,text,created_at,source,reply_id, reply_user_id,retweet_id, retweet_user_id,quote_id,quote_user_id,reply_count,retweet_count,favorite_count,quote_count,place_id, place_name, coord,hashtags, mention_ids) VALUES (%s,%s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s)"
            cursor.execute(query2, (tweet_id,user_id,text,created_at,source,
                                    reply_id, reply_user_id,
                                    retweet_id, retweet_user_id,
                                    quote_id,quote_user_id,
                                    reply_count,retweet_count,favorite_count,quote_count,
                                    place_id, place_name, coord,
                                    hashtags, mention_ids))
            
            con.commit()
    except Error as e:
        print(e)
        print(text)
        
        #Carlota: He dejado este print, porque no era capaz de almacenar emojis por la codificacion. 
        #Estoy casi segura de que se ha arreglado, pero por si acaso
    cursor.close()
    con.close() 
    return


In [ ]:
class MyStreamListener(tweepy.StreamListener):
    
    def on_data(self,data):
        # Twitter returns data in JSON format - we need to decode it first
        try:
            decoded = json.loads(data)
            
        except Exception as e:
            print ("Error on_data: %s" % str(e)) #we don't want the listener to stop
            return True
        
        #LOCATION METADATA
        
        #En caso de estar geolocalizado guardar la geolocalizacion
        #Si esta geolocalizado dentro de un bounding box (no exacta)
        if decoded.get('place') is not None:
            place_id = decoded.get('place').get('id')
            place_name =decoded.get('place').get('name')
        else:
            place_id = 'None'
            place_name = 'None'
            
        #Si es localizacion exacta
        #Geo is deprecated, they suggest to use simply coordinates
        if decoded.get('coordinates') is not None:
            m_coord = decoded.get('coordinates')['coordinates']
            #print(m_coord)
            coord=str(m_coord[0])+';'+str(m_coord[1])
            #print(coord)
            #time.sleep(100)
            
        else:
            coord = 'None'
            #print(place_id)
        
        
        #USER METADATA
        user_name = '@' + decoded.get('user').get('screen_name') #nombre cuenta @itdUPM 
        user_id=decoded.get('user').get('id') #id de la cuenta (int)
        user_loc=decoded.get('user').get('location') 
        user_follow_count=decoded.get('user').get('followers_count')
        user_friends_count=decoded.get('user').get('friends_count')
        user_fav_count=decoded.get('user').get('favourites_count')
        user_status_count=decoded.get('user').get('statuses_count')
        
        #POST METADATA
        created_at = decoded.get('created_at') #Fecha 
        tweet_id = decoded['id'] #tweet id (int64)
        source = decoded['source'] #string source (web client, android, iphone) interesante???
     
        if decoded.get('truncated'):
            text = decoded['extended_tweet']['full_text'].replace('\n',' ')
        else: 
            text = decoded['text'].replace('\n',' ') #Contenido tweet
        

        #REPLY METADATA
        reply_id=decoded['in_reply_to_status_id']
        reply_user_id=decoded['in_reply_to_user_id']

        #RETWEET
        if decoded.get('retweeted_status') is not None:
            retweet_id = decoded['retweeted_status'] ['id']
            retweet_user_id = decoded['retweeted_status']['user']['id']
            
            if decoded['retweeted_status']['truncated']:
                text = decoded['retweeted_status']['extended_tweet']['full_text'].replace('\n',' ')
            else: 
                text = decoded['retweeted_status']['text'].replace('\n',' ') #Contenido tweet
            
            #Carlota: Si es un retweet los campos de nº de retweets favs etc vienen dentro de retweeted status
            #David: ok bien visto, he añadido el id de usuario retweeteado
            reply_count = decoded['retweeted_status']['reply_count'] #Number of times this Tweet has been replied to
            retweet_count = decoded['retweeted_status']['retweet_count'] #Number of times this Tweet has been retweeted
            favorite_count = decoded['retweeted_status']['favorite_count'] #how many times this Tweet has been liked by Twitter users.
            quote_count = decoded['retweeted_status']['quote_count']
            
            #hashtags_list=decoded.get('retweeted_status').get('entities').get('hashtags')     
            #mentions=decoded.get('retweeted_status').get('entities').get('user_mentions')
            #David: para esto hay que crear una cadena de texto recorriendo la lista, el
            #código estaba en la versión anterior...
            
            hashtags_list=decoded['retweeted_status']['entities']['hashtags']     
            mentions=decoded['retweeted_status']['entities']['user_mentions']
            
            hashtags=''
            c=0
            if len(hashtags_list)>0:
                for i in range(0, len(hashtags_list)-1):
                    mh=hashtags_list[i].get('text')
                    hashtags=hashtags+mh+';'
                    c=c+1
                mh=hashtags_list[c].get('text')
                hashtags=hashtags+str(mh)
            else:
                hashtags='None'

            mention_ids=''
            c=0
            if len(mentions)>0:
                for i in range(0, len(mentions)-1):
                    mid=mentions[i].get('id_str')
                    mention_ids=mention_ids+mid+';'#use a different separator!
                    c=c+1
                mid=mentions[c].get('id_str')
                mention_ids=mention_ids+str(mid)
            else:
                mention_ids='None'

            
            #David: esto no sé si haría falta... este justo es un retweet de un post que a su ves
            #es un quote de una noticia, osea que hay dos pasos de conexión, pero el retweet 
            #con el quote ya existe... lo guardamos pero hay que tenerlo en cuenta que es redundante
            
            #Carlota: Lo quito, porque tienes razon y no habia caido...
            
            #David. lo podemos dejar porque no son campos adicionales
                     
            if decoded['retweeted_status']['is_quote_status']:
                if 'quoted_status' not in decoded['retweeted_status']:
                    quote_id='None'
                    quote_user_id='None'  
                else:
                    quote_id=decoded['retweeted_status']['quoted_status']['id']
                    quote_user_id=decoded['retweeted_status']['quoted_status']['user']['id']
            else:
                quote_id='None'
                quote_user_id='None'
                
        else:
            
            reply_count = decoded['reply_count'] #Number of times this Tweet has been replied to
            retweet_count = decoded['retweet_count'] #Number of times this Tweet has been retweeted
            favorite_count = decoded['favorite_count'] #how many times this Tweet has been liked by Twitter users.
            quote_count = decoded['quote_count']
            retweet_id = 'None'
            retweet_user_id = 'None'
            
            if decoded['is_quote_status']:
                if 'quoted_status' not in decoded:
                    quote_id='None'
                    quote_user_id='None'  
                else:
                    quote_id=decoded['quoted_status']['id']
                    quote_user_id=decoded['quoted_status']['user']['id']
            else:
                quote_id='None'
                quote_user_id='None'
                
            hashtags_list=decoded.get('entities').get('hashtags')     
            mentions=decoded.get('entities').get('user_mentions')
            
            hashtags=''
            c=0
            if len(hashtags_list)>0:
                for i in range(0, len(hashtags_list)-1):
                    mh=hashtags_list[i].get('text')
                    hashtags=hashtags+mh+';'
                    c=c+1
                mh=hashtags_list[c].get('text')
                hashtags=hashtags+str(mh)
            else:
                hashtags='None'

            mention_ids=''
            c=0
            if len(mentions)>0:
                for i in range(0, len(mentions)-1):
                    mid=mentions[i].get('id_str')
                    mention_ids=mention_ids+mid+';'#use a different separator!
                    c=c+1
                mid=mentions[c].get('id_str')
                mention_ids=mention_ids+str(mid)
            else:
                mention_ids='None'
 
        #insert data just collected into MySQL database
        connect(user_id, user_name, user_loc, user_follow_count,user_friends_count, user_fav_count,user_status_count,
                tweet_id,text,created_at,source,
                reply_id, reply_user_id,
                retweet_id,retweet_user_id,
                quote_id,quote_user_id,
                reply_count,retweet_count,favorite_count,quote_count, 
                hashtags, mention_ids,
                place_id, place_name, coord)
        
        #print("Tweet colleted at: {} ".format(str(created_at)))
        
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False

    # returning non-False reconnects the stream, with backoff.


if __name__ == '__main__':
    print ('Starting')
    #authorize twitter, initialize tweepy        
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True)
    
    #create the api and the stream object
    myStreamListener = MyStreamListener()
    myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)
    #Filter the stream by keywords
    myStream.filter(locations = campus, track=keywords)


Starting
